In [13]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix, diags, spdiags
from scipy.sparse.linalg import eigsh
import matplotlib.pyplot as plt

In [ ]:
def laplacian_eigenmaps(X, k=2, t=10, sigma=0.3):
    # Efficient nearest neighbors search
    nbrs = NearestNeighbors(n_neighbors=t + 1).fit(X)
    distances, indices = nbrs.kneighbors(X)

    m = X.shape[0]

    # Efficient weight matrix calculation
    S = np.exp(-distances ** 2 / (2 * sigma ** 2))
    W = np.zeros((m, m))
    for i in range(m):
        W[i, indices[i, 1:]] = S[i, 1:]

    # Graph Laplacian
    D = diags(np.sum(W, axis=1))
    L = D - W

    # Eigenvalue and eigenvector computation using sparse matrix
    num_eigenvalues = k + 1  # +1 for the trivial eigenvector
    eigenvalues, eigenvectors = eigsh(L, k=num_eigenvalues, which='SM')
    
    # Sort eigenvalues and eigenvectors
    sorted_indices = np.argsort(eigenvalues)
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors = eigenvectors[:, sorted_indices]

    # Project data onto the top k eigenvectors
    reduced_data = eigenvectors[:, 1:k+1]

    return reduced_data